In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

import cv2

import albumentations
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score

from tqdm import tqdm
import argparse
import os, sys, yaml

sys.path.append('/workspace/siim-rsna-2021')
from src.logger import setup_logger, LOGGER
from src.meter import mAPMeter, AUCMeter, APMeter, AverageValueMeter
from src.utils import plot_sample_images
from src.segloss import SymmetricLovaszLoss


# import neptune.new as neptune
import wandb
import pydicom

import time
from contextlib import contextmanager

import timm

import warnings

target_columns = [
    "Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance", "is_none"
]


@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [2]:
preds0 = np.load('/workspace/data/external/pseudo/preds_0.npy')
masks0 = np.load('/workspace/data/external/pseudo/masks_0.npy')

preds1 = np.load('/workspace/data/external/pseudo/preds_1.npy')
masks1 = np.load('/workspace/data/external/pseudo/masks_1.npy')


In [3]:
preds = preds0 / 3 + 2 * preds1 / 3
masks = masks0 / 3 + 2 * masks1 / 3

In [8]:
df_ext = pd.read_csv('/workspace/data/external/df_ext.csv')
df_ext.head()

,npy_path,external,cv
0,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,0
1,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,1
2,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,2
3,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,3
4,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,4


In [10]:
df_ext.iloc[0].npy_path

'/workspace/data/external/bimcv/covid19_posi_dump/sub-S10815_ses-E21412_run-1_bp-chest_vp-ap_dx.png'

In [49]:
mask_path_list = []
img_id_list = []
for i in range(len(df_ext)):
    row = df_ext.iloc[i]
    mask = masks[i]
    
    img_id = row.npy_path.split('/')[-1].split(".")[0]
    mask_npy_path = img_id + ".npy"
    save_path = "/workspace/data/external/pseudo/mask/" + mask_npy_path
    np.save(save_path, mask)
    mask_path_list.append(save_path)
    img_id_list.append(img_id)

In [50]:
df_ext["mask_path"] = mask_path_list
df_ext["image_id"] = img_id_list

In [51]:
df_pred = pd.DataFrame(preds, columns=target_columns)

In [52]:
df_ext_pred = pd.concat([df_ext, df_pred], axis=1)

In [53]:
df_ext_pred

,npy_path,external,cv,mask_path,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,is_none
0,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,0,/workspace/data/external/pseudo/mask/sub-S1081...,sub-S10815_ses-E21412_run-1_bp-chest_vp-ap_dx,0.019889,0.575185,0.210305,0.183234,0.042968
1,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,1,/workspace/data/external/pseudo/mask/sub-S0949...,sub-S09496_ses-E25324_run-1_bp-chest_vp-ap_dx,0.002663,0.878904,0.091189,0.022784,0.004981
2,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,2,/workspace/data/external/pseudo/mask/sub-S1208...,sub-S12082_ses-E24895_run-1_bp-chest_vp-ll_cr,0.329397,0.252140,0.274867,0.150027,0.377889
3,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,3,/workspace/data/external/pseudo/mask/sub-S0932...,sub-S09320_ses-E20473_run-1_bp-chest_vp-ap_cr,0.005181,0.839860,0.127447,0.030936,0.007732
4,/workspace/data/external/bimcv/covid19_posi_du...,bimcv,4,/workspace/data/external/pseudo/mask/sub-S0307...,sub-S03077_ses-E16235_acq-2_run-1_bp-chest_vp-...,0.011185,0.735432,0.192181,0.050826,0.015996
...,...,...,...,...,...,...,...,...,...,...
17674,/workspace/data/external/record/MIDRC-RICORD/M...,record,4,/workspace/data/external/pseudo/mask/419639-00...,419639-000758_08-17-2003-XR CHEST 1 VIEW AP-98...,0.001781,0.888404,0.090397,0.018905,0.003320
17675,/workspace/data/external/record/MIDRC-RICORD/M...,record,0,/workspace/data/external/pseudo/mask/SITE2-000...,SITE2-000078_01-21-2006-37750_1,0.355818,0.347745,0.243571,0.072762,0.354368
17676,/workspace/data/external/record/MIDRC-RICORD/M...,record,1,/workspace/data/external/pseudo/mask/419639-00...,419639-001214_12-27-2005-XR CHEST 1 VIEW AP-44...,0.015424,0.684383,0.261809,0.041208,0.017775
17677,/workspace/data/external/record/MIDRC-RICORD/M...,record,2,/workspace/data/external/pseudo/mask/419639-00...,419639-000481_02-16-2000-XR CHEST 1 VIEW AP-91...,0.004087,0.807572,0.158202,0.033647,0.005834


In [54]:
df_ext_pred.to_csv('/workspace/data/external/df_ext_w_pred.csv', index=False)

In [55]:
df_original = pd.read_csv("/workspace/data/df_train_study_level_npy640_3_w_bbox.csv")

In [56]:
df_original["external"] = "original"

In [57]:
df_original

,id,x,y,w,h,study_id,image_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,is_none,external
0,04f41a8958f7_image,688.06282,966.82563,518.48212,1130.17438,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_2/04f41a8958f7.png,0,original
1,04f41a8958f7_image,2482.36026,1636.77436,652.47168,652.47168,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_2/04f41a8958f7.png,0,original
2,04f41a8958f7_image,1235.67308,1628.03597,509.74353,559.26147,6e4a0581cefe,04f41a8958f7,1,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0,1,0,0,0,/workspace/data/train_640_2/04f41a8958f7.png,0,original
3,0572ef0d0c1a_image,1818.65264,233.50598,613.04395,839.53784,adbfed2da701,0572ef0d0c1a,1,/workspace/data/train/adbfed2da701/e2fa197720c...,0,1,0,0,3,/workspace/data/train_640_2/0572ef0d0c1a.png,0,original
4,0572ef0d0c1a_image,598.60492,61.37052,688.54175,881.81674,adbfed2da701,0572ef0d0c1a,1,/workspace/data/train/adbfed2da701/e2fa197720c...,0,1,0,0,3,/workspace/data/train_640_2/0572ef0d0c1a.png,0,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9671,ffd9b6cf2961_image,707.25199,722.07926,392.14044,849.18683,7eed9af03814,ffd9b6cf2961,1,/workspace/data/train/7eed9af03814/668a64e3a5f...,0,1,0,0,1,/workspace/data/train_640_2/ffd9b6cf2961.png,0,original
9672,ffdc682f7680_image,2729.27083,332.26044,1496.25016,2604.58334,a0cb0b96fb3d,ffdc682f7680,1,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0,1,0,0,2,/workspace/data/train_640_2/ffdc682f7680.png,0,original
9673,ffdc682f7680_image,1005.81250,1584.67711,662.22913,775.83337,a0cb0b96fb3d,ffdc682f7680,1,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0,1,0,0,2,/workspace/data/train_640_2/ffdc682f7680.png,0,original
9674,ffe942c8655f_image,208.86463,91.53448,450.96747,628.05473,7d82d53204b8,ffe942c8655f,1,/workspace/data/train/7d82d53204b8/431ee249d16...,0,1,0,0,1,/workspace/data/train_640_2/ffe942c8655f.png,0,original


In [58]:
df_concat = pd.concat([df_original, df_ext_pred], axis=0, ignore_index=True)

In [59]:
df_concat

,id,x,y,w,h,study_id,image_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,is_none,external,mask_path
0,04f41a8958f7_image,688.06282,966.82563,518.48212,1130.17438,6e4a0581cefe,04f41a8958f7,1.0,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0.000000,1.000000,0.000000,0.000000,0,/workspace/data/train_640_2/04f41a8958f7.png,0.000000,original,NaN
1,04f41a8958f7_image,2482.36026,1636.77436,652.47168,652.47168,6e4a0581cefe,04f41a8958f7,1.0,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0.000000,1.000000,0.000000,0.000000,0,/workspace/data/train_640_2/04f41a8958f7.png,0.000000,original,NaN
2,04f41a8958f7_image,1235.67308,1628.03597,509.74353,559.26147,6e4a0581cefe,04f41a8958f7,1.0,/workspace/data/train/6e4a0581cefe/018ed20fa9c...,0.000000,1.000000,0.000000,0.000000,0,/workspace/data/train_640_2/04f41a8958f7.png,0.000000,original,NaN
3,0572ef0d0c1a_image,1818.65264,233.50598,613.04395,839.53784,adbfed2da701,0572ef0d0c1a,1.0,/workspace/data/train/adbfed2da701/e2fa197720c...,0.000000,1.000000,0.000000,0.000000,3,/workspace/data/train_640_2/0572ef0d0c1a.png,0.000000,original,NaN
4,0572ef0d0c1a_image,598.60492,61.37052,688.54175,881.81674,adbfed2da701,0572ef0d0c1a,1.0,/workspace/data/train/adbfed2da701/e2fa197720c...,0.000000,1.000000,0.000000,0.000000,3,/workspace/data/train_640_2/0572ef0d0c1a.png,0.000000,original,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27350,NaN,NaN,NaN,NaN,NaN,NaN,419639-000758_08-17-2003-XR CHEST 1 VIEW AP-98...,NaN,NaN,0.001781,0.888404,0.090397,0.018905,4,/workspace/data/external/record/MIDRC-RICORD/M...,0.003320,record,/workspace/data/external/pseudo/mask/419639-00...
27351,NaN,NaN,NaN,NaN,NaN,NaN,SITE2-000078_01-21-2006-37750_1,NaN,NaN,0.355818,0.347745,0.243571,0.072762,0,/workspace/data/external/record/MIDRC-RICORD/M...,0.354368,record,/workspace/data/external/pseudo/mask/SITE2-000...
27352,NaN,NaN,NaN,NaN,NaN,NaN,419639-001214_12-27-2005-XR CHEST 1 VIEW AP-44...,NaN,NaN,0.015424,0.684383,0.261809,0.041208,1,/workspace/data/external/record/MIDRC-RICORD/M...,0.017775,record,/workspace/data/external/pseudo/mask/419639-00...
27353,NaN,NaN,NaN,NaN,NaN,NaN,419639-000481_02-16-2000-XR CHEST 1 VIEW AP-91...,NaN,NaN,0.004087,0.807572,0.158202,0.033647,2,/workspace/data/external/record/MIDRC-RICORD/M...,0.005834,record,/workspace/data/external/pseudo/mask/419639-00...


In [60]:
df_concat.to_csv('/workspace/data/df_train_w_pseudo.csv', index=False)

In [32]:
df_concat.groupby('image_id').first()

,id,x,y,w,h,study_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,is_none,external,mask_path
image_id,,,,,,,,,,,,,,,,,
000a312787f2,000a312787f2_image,789.28836,582.43035,1026.65662,1917.30292,5776db0cec75,1.0,/workspace/data/train/5776db0cec75/81456c9c542...,0.000000,1.000000,0.000000,0.000000,0,/workspace/data/train_640_2/000a312787f2.png,0.000000,original,None
000c3a3f293f,000c3a3f293f_image,NaN,NaN,NaN,NaN,ff0879eb20ed,0.0,/workspace/data/train/ff0879eb20ed/d8a644cc4f9...,1.000000,0.000000,0.000000,0.000000,0,/workspace/data/train_640_2/000c3a3f293f.png,1.000000,original,None
0012ff7358bc,0012ff7358bc_image,677.42216,197.97662,867.79767,999.78214,9d514ce429a7,1.0,/workspace/data/train/9d514ce429a7/22897cd1daa...,0.000000,1.000000,0.000000,0.000000,1,/workspace/data/train_640_2/0012ff7358bc.png,0.000000,original,None
001398f4ff4f,001398f4ff4f_image,2729.00000,2181.33331,948.00012,604.00000,28dddc8559b2,1.0,/workspace/data/train/28dddc8559b2/4d47bc042ee...,0.000000,0.000000,0.000000,1.000000,1,/workspace/data/train_640_2/001398f4ff4f.png,0.000000,original,None
001bd15d1891,001bd15d1891_image,623.23328,1050.00000,714.00000,1106.00000,dfd9fdd85a3e,1.0,/workspace/data/train/dfd9fdd85a3e/49170afa4f2...,0.000000,1.000000,0.000000,0.000000,3,/workspace/data/train_640_2/001bd15d1891.png,0.000000,original,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffd91a2c4ca0,ffd91a2c4ca0_image,NaN,NaN,NaN,NaN,8332bdaddb6e,0.0,/workspace/data/train/8332bdaddb6e/f346cda52da...,1.000000,0.000000,0.000000,0.000000,4,/workspace/data/train_640_2/ffd91a2c4ca0.png,1.000000,original,None
ffd9b6cf2961,ffd9b6cf2961_image,2197.38566,841.07361,316.41699,451.63758,7eed9af03814,1.0,/workspace/data/train/7eed9af03814/668a64e3a5f...,0.000000,1.000000,0.000000,0.000000,1,/workspace/data/train_640_2/ffd9b6cf2961.png,0.000000,original,None
ffdc682f7680,ffdc682f7680_image,2729.27083,332.26044,1496.25016,2604.58334,a0cb0b96fb3d,1.0,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0.000000,1.000000,0.000000,0.000000,2,/workspace/data/train_640_2/ffdc682f7680.png,0.000000,original,None


In [33]:
df_original.groupby('image_id').first()

,id,x,y,w,h,study_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,is_none,external
image_id,,,,,,,,,,,,,,,,
000a312787f2,000a312787f2_image,789.28836,582.43035,1026.65662,1917.30292,5776db0cec75,1,/workspace/data/train/5776db0cec75/81456c9c542...,0,1,0,0,0,/workspace/data/train_640_2/000a312787f2.png,0,original
000c3a3f293f,000c3a3f293f_image,NaN,NaN,NaN,NaN,ff0879eb20ed,0,/workspace/data/train/ff0879eb20ed/d8a644cc4f9...,1,0,0,0,0,/workspace/data/train_640_2/000c3a3f293f.png,1,original
0012ff7358bc,0012ff7358bc_image,677.42216,197.97662,867.79767,999.78214,9d514ce429a7,1,/workspace/data/train/9d514ce429a7/22897cd1daa...,0,1,0,0,1,/workspace/data/train_640_2/0012ff7358bc.png,0,original
001398f4ff4f,001398f4ff4f_image,2729.00000,2181.33331,948.00012,604.00000,28dddc8559b2,1,/workspace/data/train/28dddc8559b2/4d47bc042ee...,0,0,0,1,1,/workspace/data/train_640_2/001398f4ff4f.png,0,original
001bd15d1891,001bd15d1891_image,623.23328,1050.00000,714.00000,1106.00000,dfd9fdd85a3e,1,/workspace/data/train/dfd9fdd85a3e/49170afa4f2...,0,1,0,0,3,/workspace/data/train_640_2/001bd15d1891.png,0,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffcc6edd9445,ffcc6edd9445_image,NaN,NaN,NaN,NaN,7e6c68462e06,0,/workspace/data/train/7e6c68462e06/98011bca5fd...,1,0,0,0,1,/workspace/data/train_640_2/ffcc6edd9445.png,1,original
ffd91a2c4ca0,ffd91a2c4ca0_image,NaN,NaN,NaN,NaN,8332bdaddb6e,0,/workspace/data/train/8332bdaddb6e/f346cda52da...,1,0,0,0,4,/workspace/data/train_640_2/ffd91a2c4ca0.png,1,original
ffd9b6cf2961,ffd9b6cf2961_image,2197.38566,841.07361,316.41699,451.63758,7eed9af03814,1,/workspace/data/train/7eed9af03814/668a64e3a5f...,0,1,0,0,1,/workspace/data/train_640_2/ffd9b6cf2961.png,0,original


In [48]:
np.load(df_concat.mask_path.iloc[-1])

array('SITE2-000028_01-22-2001-53390_1', dtype='<U31')